# QTL data postprocessing

In [ ]:
#!/usr/bin/env Rscript

# eQTL Hierarchical Multiple Testing Framework
# ===============================================
# Implements a three-step procedure:
# 1. Local adjustment: p-values of all cis-SNPs adjusted within each gene
# 2. Global adjustment: minimum adjusted p-values from Step 1 further adjusted across all genes
# 3. Identification of significant eSNPs: SNPs with locally adjusted p-value below the threshold

# Load required libraries
suppressPackageStartupMessages({
  library(data.table)
  library(qvalue)
  library(dplyr)
  library(tidyr)
})

# Utility function for NULL coalescing
`%||%` <- function(a, b) if (is.null(a)) b else a

#===================================
# Parameter Processing
#===================================
parse_args <- function() {
  args <- commandArgs(TRUE)
  params <- list()
  
  # Process key=value pairs
  for (arg in args) {
    if (grepl("=", arg)) {
      pair <- strsplit(arg, "=")[[1]]
      params[[pair[1]]] <- pair[2]
    }
  }
  
  # Set defaults if not provided
  params$workdir <- params$workdir %||% getwd()
  params$qtl_type <- params$qtl_type %||% "LR"
  params$maf_cutoff <- as.numeric(params$maf_cutoff %||% 0.05)
  params$cis_window <- as.numeric(params$cis_window %||% 1000000)
  params$pvalue_cutoff <- as.numeric(params$pvalue_cutoff %||% 0.05)
  params$fdr_threshold <- as.numeric(params$fdr_threshold %||% 0.05)
  params$gene_coordinates <- params$gene_coordinates %||% "/home/al4225/project/resource/look_up_gene_id.tsv"
  params$output_dir <- params$output_dir %||% file.path("/home/al4225/output", basename(params$workdir))
  params$archive_dir <- params$archive_dir %||% file.path(params$workdir, "archived")
  
  # File patterns
  params$regional_pattern <- params$regional_pattern %||% "*.cis_qtl_regional.fdr.gz$"
  params$n_variants_pattern <- params$n_variants_pattern %||% "*.n_variants_stats.txt.gz$"
  params$pair_pattern <- params$pair_pattern %||% "*.cis_qtl.pairs.tsv.gz$"
  
  return(params)
}

#===================================
# Data Loaders
#===================================

load_regional_data <- function(params) {
  setwd(params$workdir)
  
  # First try to find permutation-based files
  files <- list.files(pattern = params$regional_pattern, full.names = TRUE)
  
  if (length(files) > 0) {
    message("Loading regional data with permutation tests")
    data <- rbindlist(lapply(files, fread))
    return(list(data = data, files = files, has_permutation = TRUE))
  }
  
  # If no permutation files, check for n_variants files in current directory
  n_variants_files <- list.files(pattern = params$n_variants_pattern, full.names = TRUE)
  if (length(n_variants_files) > 0) {
    message("Loading n_variants statistics from current directory")
    data <- rbindlist(lapply(n_variants_files, fread))
    return(list(data = data, files = n_variants_files, has_permutation = FALSE))
  }
  
  message("No regional data found")
  return(NULL)
}

# Column name extractor - handles LR vs interaction column differences
extract_column_names <- function(file_path, qtl_type) {
  header <- system(paste0("zcat ", file_path, " | head -1"), intern = TRUE)
  cols <- strsplit(header, "\t")[[1]]
  
  column_info <- list(
    all_columns = cols
  )
  
  # Determine p-value column
  if (qtl_type == "interaction") {
    interaction_cols <- grep("pvalue_.*_interaction", cols, value = TRUE)
    column_info$p_col <- if (length(interaction_cols) > 0) interaction_cols[1] else "pvalue"
  } else {
    column_info$p_col <- "pvalue"
  }
  
  # Find q-value column if it exists
  if (qtl_type == "interaction") {
    qval_cols <- grep("qvalue_.*_interaction", cols, value = TRUE)
    column_info$q_col <- if (length(qval_cols) > 0) qval_cols[1] else "qvalue"
  } else {
    column_info$q_col <- "qvalue"
  }
  
  column_info$p_idx <- which(cols == column_info$p_col)
  if (length(column_info$p_idx) == 0) {
    stop(sprintf("P-value column '%s' not found", column_info$p_col))
  }
  
  return(column_info)
}

# Pair data loader with conditional filtering
load_pair_data <- function(params) {
  setwd(params$workdir)
  
  files <- list.files(pattern = params$pair_pattern, full.names = TRUE)
  if (length(files) == 0) {
    stop("No pair files found")
  }
  
  # Get column names
  column_info <- extract_column_names(files[1], params$qtl_type)
  
  # Only filter if pvalue_cutoff is less than 1
  filter_by_p <- params$pvalue_cutoff < 1
  
  if (filter_by_p) {
    # Filter rows with p-value < threshold for efficiency
    files_str <- paste(files, collapse = " ")
    awk_cmd <- sprintf("awk 'NR==1 {print; next} $%d < %s'", 
                       column_info$p_idx, params$pvalue_cutoff)
    cmd <- sprintf("zcat %s | %s", files_str, awk_cmd)
    
    message(sprintf("Filtering pair data with p-value < %g", params$pvalue_cutoff))
    data <- fread(cmd)
  } else {
    # Load all data (may be huge)
    message("Loading all pair data (no p-value filtering)")
    files_str <- paste(files, collapse = " ")
    cmd <- sprintf("zcat %s", files_str)
    data <- fread(cmd=cmd)
  }
  
  return(list(
    data = data, 
    files = files,
    column_info = column_info
  ))
}

# Gene coordinate data loader
load_gene_coordinates <- function(params) {
  gene_coords <- fread(params$gene_coordinates)
  message(sprintf("Loaded gene coordinates with %d entries", nrow(gene_coords)))
  return(gene_coords)
}

# Calculate feature positions for cis-window filtering
calculate_feature_positions <- function(pair_data, gene_coords, cis_window) {
  # Extract ENSEMBL IDs from molecular_trait_object_id
  extract_ensembl <- function(ids) {
    pattern <- "^.*?(ENSG\\d+).*$"
    ensembl_ids <- gsub(pattern, "\\1", ids)
    not_matched <- ensembl_ids == ids & !grepl("ENSG\\d+", ids)
    if (any(not_matched)) {
      ensembl_ids[not_matched] <- NA
    }
    return(ensembl_ids)
  }
  
  # Get unique molecular traits
  unique_traits <- pair_data %>%
    select(molecular_trait_object_id, chrom) %>%
    distinct() %>%
    mutate(ensembl_id = extract_ensembl(molecular_trait_object_id))
  
  # Prepare lookup data (remove chr prefix if present)
  lookup_for_join <- gene_coords %>%
    mutate(chrom_num = gsub("^chr", "", chr))
  
  # Join with lookup to get TSS/TES positions
  merged_traits <- unique_traits %>%
    left_join(
      lookup_for_join %>% 
        select(chrom = chrom_num, gene_id, gene_start = start, gene_end = end),
      by = c("ensembl_id" = "gene_id", "chrom" = "chrom")
    )
  
  # For unmapped genes, calculate approximate positions from variant data
  if (sum(is.na(merged_traits$gene_start)) > 0) {
    fallback_positions <- pair_data %>%
      filter(molecular_trait_object_id %in% 
            merged_traits$molecular_trait_object_id[is.na(merged_traits$gene_start)]) %>%
      group_by(molecular_trait_object_id, chrom) %>%
      summarize(
        approx_tss = first(pos) - first(start_distance),
        approx_tes = first(pos) - first(end_distance),
        .groups = 'drop'
      )
      
    merged_traits <- merged_traits %>%
      left_join(fallback_positions, by = c("molecular_trait_object_id", "chrom")) %>%
      mutate(
        feature_tss = coalesce(gene_start, approx_tss),
        feature_tes = coalesce(gene_end, approx_tes)
      )
  } else {
    merged_traits <- merged_traits %>%
      mutate(
        feature_tss = gene_start,
        feature_tes = gene_end
      )
  }
    
  # Add cis window range
  feature_positions <- merged_traits %>%
    mutate(
      cis_start = feature_tss - cis_window,
      cis_end = feature_tes + cis_window
    ) %>%
    select(molecular_trait_object_id, chrom, feature_tss, feature_tes, cis_start, cis_end)
    
  return(feature_positions)
}

# Helper function for filtering variants
apply_variant_filtering <- function(pair_data, gene_coords, params) {
  # Calculate feature positions for cis-window filtering
  feature_positions <- calculate_feature_positions(pair_data, gene_coords, params$cis_window)
  
  # Get original counts per gene
  original_counts <- pair_data %>%
    group_by(molecular_trait_object_id) %>%
    summarize(
      original_count = n(),
      original_n_variants = first(n_variants),
      .groups = "drop"
    )
  
  # Apply both MAF and cis-window filtering
  filtered_data <- pair_data %>%
    left_join(
      feature_positions %>% 
        select(molecular_trait_object_id, cis_start, cis_end),
      by = "molecular_trait_object_id"
    ) %>%
    filter(pmin(af, 1-af) > params$maf_cutoff) %>%
    filter(pos >= cis_start & pos <= cis_end)
  
  # Calculate filtered counts per gene
  filtered_counts <- filtered_data %>%
    group_by(molecular_trait_object_id) %>%
    summarize(
      filtered_count = n(),
      .groups = "drop"
    )
  
  # Compute ratios and new n_variants
  n_variants_stats <- inner_join(original_counts, filtered_counts, 
                                by = "molecular_trait_object_id") %>%
    mutate(
      ratio = filtered_count / original_count,
      new_n_variants = round(original_n_variants * ratio)
    )
  
  message(sprintf("Average filtering ratio: %.3f", mean(n_variants_stats$ratio)))
  
  return(list(
    filtered_data = filtered_data,
    n_variants_stats = n_variants_stats
  ))
}

#===================================
# Step 1: Local Adjustment Interface
#===================================

# Apply local adjustment methods 
# Returns a new list with locally adjusted values
perform_local_adjustment <- function(data, params, method) {
  if (method == "permutation") {
    return(permutation_local_adjustment(data, params))
  } else if (method == "bonferroni") {
    is_filtered <- !is.null(params$filtered) && params$filtered
    return(bonferroni_local_adjustment(data, params, is_filtered))
  } else {
    stop(sprintf("Unknown local adjustment method: %s", method))
  }
}

# Permutation-based local adjustment
permutation_local_adjustment <- function(data, params) {
  message("Using permutation-based local adjustment")
  
  if (is.null(data$regional_data) || is.null(data$regional_data$data)) {
    stop("Permutation local adjustment requires regional data")
  }
  
  # Extract the gene-level locally adjusted p-values from regional data
  regional_data <- data$regional_data$data
  
  # Check if we have the required columns
  has_p_perm <- "p_perm" %in% colnames(regional_data)
  has_p_beta <- "p_beta" %in% colnames(regional_data)
  
  if (!has_p_perm && !has_p_beta) {
    stop("Permutation local adjustment requires p_perm or p_beta columns in regional data")
  }
  
  # Create a unified output structure containing gene-level locally adjusted p-values
  gene_level_pvalues <- regional_data %>%
    select(molecular_trait_object_id) %>%
    distinct()
  
  # Add all available adjusted p-value columns to create unified interface
  for (col in c("p_perm", "p_beta")) {
    if (col %in% colnames(regional_data)) {
      gene_level_pvalues[[col]] <- regional_data[[col]]
    }
  }
  
  # Create new result data structure with locally adjusted values
  result <- list()
  for (key in names(data)) {
    result[[key]] <- data[[key]]  # Copy existing data
  }
  
  # Add local adjustment information
  result$gene_level_pvalues <- gene_level_pvalues
  result$local_adjustment_info <- list(
    method = "permutation",
    p_value_columns = c(
      if("p_perm" %in% colnames(gene_level_pvalues)) "p_perm",
      if("p_beta" %in% colnames(gene_level_pvalues)) "p_beta"
    ),
    is_filtered = FALSE
  )
  
  return(result)
}

# Bonferroni-based local adjustment
bonferroni_local_adjustment <- function(data, params, is_filtered = FALSE) {
  message(sprintf("Applying bonferroni local adjustment (filtered = %s)", is_filtered))
  
  pair_data <- data$pair_data$data
  p_col <- data$pair_data$column_info$p_col
  
  # Create new result data structure
  result <- list()
  for (key in names(data)) {
    result[[key]] <- data[[key]]  # Copy existing data
  }

  if (!is_filtered && !"n_variants" %in% colnames(pair_data)) {
    message("n_variants not found. Counting SNPs per gene, which may overestimate tests if data is pre-filtered.")
  }
    
  # Apply filtering if needed
  if (is_filtered) {
    # Apply filtering and calculate new n_variants
    filtered_results <- apply_variant_filtering(pair_data, data$gene_coords, params)
    filtered_pair_data <- filtered_results$filtered_data
    result$n_variants_stats <- filtered_results$n_variants_stats
    
    # Add bonferroni adjustments
    adjusted_pair_data <- filtered_pair_data %>%
      left_join(
        filtered_results$n_variants_stats %>% 
          select(molecular_trait_object_id, new_n_variants),
        by = "molecular_trait_object_id"
      ) %>%
      mutate(p_bonferroni_adj = pmin(1, !!sym(p_col) * new_n_variants))
  } else {
    # For original version, use existing n_variants
    if ("n_variants" %in% colnames(pair_data)) {
      adjusted_pair_data <- pair_data %>%
        mutate(p_bonferroni_adj = pmin(1, !!sym(p_col) * n_variants))
    } else {
      # If n_variants not available, count per gene
      adjusted_pair_data <- pair_data %>%
        group_by(molecular_trait_object_id) %>%
        mutate(p_bonferroni_adj = pmin(1, !!sym(p_col) * n())) %>%
        ungroup()
    }
  }
  
  # Update pair data with locally adjusted p-values
  result$pair_data$data <- adjusted_pair_data
  
  # Extract gene-level minimum p-values
  gene_level_pvalues <- adjusted_pair_data %>%
    group_by(molecular_trait_object_id) %>%
    summarize(p_bonferroni_min = min(p_bonferroni_adj)) %>%
    ungroup()
  
  # Set consistent interface output
  result$gene_level_pvalues <- gene_level_pvalues
  result$local_adjustment_info <- list(
    method = "bonferroni",
    p_value_columns = "p_bonferroni_min",
    is_filtered = is_filtered
  )
  
  return(result)
}

#===================================
# Step 2: Global Adjustment Module  
#===================================

# Apply global adjustment
# Returns a new list with globally adjusted values added
perform_global_adjustment <- function(data, params, method = "fdr") {
  message(sprintf("Applying %s global adjustment", method))
  
  # Validate we have gene-level p-values from Step 1
  if (is.null(data$gene_level_pvalues) || is.null(data$local_adjustment_info)) {
    stop("Cannot perform global adjustment: no gene-level p-values found")
  }
  
  # Create new result data structure
  result <- list()
  for (key in names(data)) {
    result[[key]] <- data[[key]]  # Copy existing data
  }
  
  # Get the gene-level p-values and metadata
  gene_pvalues <- data$gene_level_pvalues
  p_columns <- data$local_adjustment_info$p_value_columns
  local_method <- data$local_adjustment_info$method
  is_filtered <- data$local_adjustment_info$is_filtered
  
  # Create a copy of gene_pvalues for modification
  adjusted_gene_pvalues <- gene_pvalues
  
  # Apply the selected global method to each gene-level p-value column
  result_columns <- c()
  
  for (p_col in p_columns) {
    # Create column name for the adjusted values
    if (method == "fdr") {
      adjusted_col <- paste0("fdr_", p_col)
    } else if (method == "qvalue") {
      adjusted_col <- paste0("q_", p_col)
    } else {
      stop(sprintf("Unknown global adjustment method: %s", method))
    }
    
    # Apply the adjustment
    if (method == "fdr") {
      adjusted_gene_pvalues[[adjusted_col]] <- p.adjust(adjusted_gene_pvalues[[p_col]], method = "fdr")
    } else if (method == "qvalue") {
      adjusted_gene_pvalues[[adjusted_col]] <- qvalue(adjusted_gene_pvalues[[p_col]])$qvalues
    }
    
    result_columns <- c(result_columns, adjusted_col)
  }
  
  # Store the globally adjusted values
  result$global_adjustment_results <- list(
    gene_pvalues = adjusted_gene_pvalues,
    local_method = local_method,
    global_method = method,
    p_value_columns = p_columns,
    result_columns = result_columns,
    is_filtered = is_filtered
  )
  
  # If regional data exists, add global adjustment results to it
  if (!is.null(result$regional_data) && !is.null(result$regional_data$data)) {
    updated_regional_data <- result$regional_data$data %>%
      left_join(
        adjusted_gene_pvalues %>% select(molecular_trait_object_id, all_of(result_columns)),
        by = "molecular_trait_object_id"
      )
    
    result$regional_data$data <- updated_regional_data
  }
  
  return(result)
}

#===================================
# Step 3: Significant SNP Identification
#===================================

# Identify significant SNPs without modifying the input data
# Returns a new list with significant SNPs added
identify_significant_snps <- function(data, params, global_informed_local_method) {
  message(sprintf("Identifying significant SNPs using %s method", global_informed_local_method))
  
  if (global_informed_local_method == "permutation") {
    result <- identify_permutation_snps(data, params)
  } else if (global_informed_local_method == "bonferroni") {
    result <- identify_bonferroni_snps(data, params)
  } else if (global_informed_local_method == "qvalue") {
    result <- identify_qvalue_snps(data, params)
  } else {
    stop(sprintf("Unknown global_informed_local_method: %s", global_informed_local_method))
  }
  
  return(result)
}

# Identify significant SNPs using permutation thresholds
identify_permutation_snps <- function(data, params, gene_pvalues, result_columns, p_value_columns) {
  # Use beta-based results if available, otherwise use permutation results
  beta_result_col <- NULL
  beta_pvalue_col <- NULL
  
  for (i in seq_along(result_columns)) {
    if (grepl("beta", result_columns[i])) {
      beta_result_col <- result_columns[i]
      beta_pvalue_col <- p_value_columns[i]
      break
    }
  }
  
  # If no beta columns, use the first available
  if (is.null(beta_result_col) && length(result_columns) > 0) {
    beta_result_col <- result_columns[1]
    beta_pvalue_col <- p_value_columns[1]
  }
  
  if (is.null(beta_result_col)) {
    warning("No suitable global adjustment column found")
    return(data)
  }
  
  # Find significant genes
  significant_genes <- gene_pvalues %>%
    filter(!!sym(beta_result_col) < params$fdr_threshold)
  
  if (nrow(significant_genes) == 0) {
    message(sprintf("No significant genes found using %s threshold", beta_result_col))
    return(data)
  }
  
  # Only proceed if beta shape parameters are available in regional data
  regional_data <- data$regional_data$data
  if (!all(c("beta_shape1", "beta_shape2") %in% colnames(regional_data))) {
    warning("Cannot identify significant SNPs - beta shape parameters not available")
    return(data)
  }
  
  # Find the threshold p-value
  max_passing <- max(significant_genes[[beta_pvalue_col]], na.rm = TRUE)
  
  # Find the minimum non-passing value (if any)
  non_passing <- gene_pvalues %>%
    filter(!!sym(beta_result_col) >= params$fdr_threshold) %>%
    filter(!is.na(!!sym(beta_pvalue_col))) %>%
    arrange(!!sym(beta_pvalue_col))
  
  # Calculate the threshold
  if (nrow(non_passing) > 0) {
    threshold <- (max_passing + non_passing[[beta_pvalue_col]][1]) / 2
  } else {
    threshold <- max_passing
  }
  
  # Create a copy of the regional data with nominal thresholds added
  updated_regional_data <- regional_data %>%
    mutate(pval_nominal_threshold = qbeta(threshold, beta_shape1, beta_shape2))
  
  # Update regional data
  data$regional_data$data <- updated_regional_data
  
  # Find significant SNPs using the nominal thresholds
  significant_pairs <- data$pair_data$data %>%
    left_join(
      updated_regional_data %>% 
        select(molecular_trait_object_id, pval_nominal_threshold),
      by = "molecular_trait_object_id"
    ) %>%
    filter(!!sym(data$pair_data$column_info$p_col) < pval_nominal_threshold)
  
  # Store results
  result_name <- "permutation_significant"
  data$significant_qtls[[result_name]] <- significant_pairs
  
  message(sprintf("Identified %d significant SNPs using permutation threshold", 
                  nrow(significant_pairs)))
  
  return(data)
}

# Identify significant SNPs using bonferroni thresholds
identify_bonferroni_snps <- function(data, params, gene_pvalues, result_columns, p_value_columns) {
  # Use the first result column
  if (length(result_columns) == 0 || length(p_value_columns) == 0) {
    warning("No global adjustment result columns found")
    return(data)
  }
  
  result_col <- result_columns[1]
  p_value_col <- p_value_columns[1]
  is_filtered <- data$global_adjustment_results$is_filtered
  global_method <- data$global_adjustment_results$global_method
  
  # Find genes that pass the threshold
  significant_genes <- gene_pvalues %>% 
    filter(!!sym(result_col) < params$fdr_threshold)
  
  if (nrow(significant_genes) == 0) {
    message(sprintf("No significant genes identified at %s threshold %g", 
                    result_col, params$fdr_threshold))
    return(data)
  }
  
  # Get the maximum locally-adjusted p-value that passes the global threshold
  threshold <- max(significant_genes[[p_value_col]], na.rm = TRUE)
  
  # Find SNPs with locally-adjusted p-values below this threshold
  significant_pairs <- data$pair_data$data %>%
    filter(p_bonferroni_adj <= threshold)
  
  # Determine result name based on context
  result_name <- sprintf("bonferroni_%s_%s", 
                         global_method,
                         if(is_filtered) "filtered" else "original")
  
  data$significant_qtls[[result_name]] <- significant_pairs
  
  message(sprintf("Identified %d significant SNPs using bonferroni/%s threshold (%g)", 
                  nrow(significant_pairs), global_method, threshold))
  
  return(data)
}

# Identify significant SNPs using q-value approach per gene
identify_qvalue_snps <- function(data, params, gene_pvalues, result_columns) {
  message("Identifying significant SNPs using q-value per gene method")
  
  # Use the first globally adjusted p-value column to identify significant eGenes
  result_col <- result_columns[1]
  significant_genes <- gene_pvalues[get(result_col) < params$fdr_threshold, 
                                   molecular_trait_object_id]
  
  if (length(significant_genes) == 0) {
    message(sprintf("No significant eGenes found using %s threshold %g", 
                    result_col, params$fdr_threshold))
    return(data)
  }
  
  # Get column names from pair data
  p_col <- data$pair_data$column_info$p_col
  q_col <- data$pair_data$column_info$q_col  # Attempt to retrieve q_col
  
  # Subset pair data to significant eGenes
  pair_dt <- data$pair_data$data
  setDT(pair_dt)  # Ensure pair data is a data.table
  significant_pair_dt <- pair_dt[molecular_trait_object_id %in% significant_genes]
  
  # Check if q_col exists and is present in the data
  if (!is.null(q_col) && q_col %in% colnames(significant_pair_dt)) {
    message("Using existing q-value column:", q_col)
    # Use the existing q-values from q_col
    significant_pair_dt[, qvalue := get(q_col)]
  } else {
    message("Computing q-values for each gene's SNPs")
    # Compute q-values using the original p-values
    significant_pair_dt[, qvalue := qvalue(get(p_col))$qvalues, 
                        by = molecular_trait_object_id]
  }
  
  # Filter SNPs with q-value below the FDR threshold
  significant_snps <- significant_pair_dt[qvalue < params$fdr_threshold]
  
  # Generate a unique result name based on the global method
  global_method <- data$global_adjustment_results$global_method
  result_name <- sprintf("qvalue_per_gene_%s", global_method)
  
  # Store significant SNPs in the result structure
  if (is.null(data$significant_qtls)) {
    data$significant_qtls <- list()
  }
  data$significant_qtls[[result_name]] <- significant_snps
  
  message(sprintf("Identified %d significant SNPs using q-value per gene method with %s global adjustment", 
                  nrow(significant_snps), global_method))
  
  return(data)
}

#===================================
# Output Management
#===================================

# Create output path helper function to reduce redundancy
create_output_path <- function(output_dir, base_name, suffix) {
  file.path(output_dir, gsub("\\.tsv\\.gz$|\\.fdr\\.gz$", suffix, base_name))
}

# Archive original files
archive_original_files <- function(params) {
  workdir <- params$workdir
  
  # Find files to archive
  parquet_files <- list.files(workdir, pattern = "parquet$", full.names = TRUE)
  region_files <- list.files(workdir, pattern = "\\.*region.*$", full.names = TRUE)
  to_archive <- c(parquet_files, region_files)
  
  # Only archive files that exist
  existing_files <- to_archive[file.exists(to_archive)]
  
  if (length(existing_files) > 0) {
    archive_dir <- params$archive_dir
    if (!dir.exists(archive_dir)) dir.create(archive_dir, recursive = TRUE)
    
    sapply(existing_files, function(f) {
      file.rename(f, file.path(archive_dir, basename(f)))
    })
    
    message(sprintf("Archived %d original files", length(existing_files)))
  }
}

# Write results to output files - optimized version
write_results <- function(data, params) {
  # Create output directory if it doesn't exist
  output_dir <- params$output_dir
  if (!dir.exists(output_dir)) dir.create(output_dir, recursive = TRUE)
  
  # Generate base filenames
  if (!is.null(data$regional_data) && !is.null(data$regional_data$files)) {
    regional_base <- gsub("chr[^_]*", "chrall", basename(data$regional_data$files[[1]]))
  } else {
    regional_base <- "chrall.cis_qtl_regional.fdr.gz"
  }
  
  pair_base <- gsub("chr[^_]*", "chrall", basename(data$pair_data$files[[1]]))
  
  # Write regional data if permutation testing was done
  if (!is.null(data$regional_data) && data$regional_data$has_permutation) {
    path <- file.path(output_dir, regional_base)
    fwrite(data$regional_data$data, path, sep = "\t")
    
    # Also write permutation significant events
    if ("q_perm" %in% colnames(data$regional_data$data)) {
      perm_path <- create_output_path(output_dir, regional_base, 
                                     ".significant_events.permutation_adjusted.tsv.gz")
      
      perm_results <- data$regional_data$data %>% filter(q_perm < params$fdr_threshold)
      fwrite(perm_results, perm_path, sep = "\t")
    }
  }
  
  # Write global adjustment results if available
  if (!is.null(data$global_adjustment_results)) {
    for (method in unique(c("fdr", "qvalue"))) {
      if (any(grepl(paste0("^", method), colnames(data$global_adjustment_results$gene_pvalues)))) {
        gene_path <- create_output_path(output_dir, regional_base, 
                                      paste0(".egene_min_pvalues.", method, ".tsv.gz"))
        fwrite(data$global_adjustment_results$gene_pvalues, gene_path, sep = "\t")
      }
    }
  }
  
# Write significant QTLs
  for (type in names(data$significant_qtls)) {
    if (nrow(data$significant_qtls[[type]]) > 0) {
      # Construct output filename based on the type
      if (grepl("^perm", type)) {
        outfile <- ".significant_qtl.permutation_adjusted.tsv.gz"
      } else {
        # For bonferroni results, parse the type string
        parts <- strsplit(type, "_")[[1]]
        method <- parts[2]  # fdr or qvalue
        filtered <- parts[3]  # original or filtered
        
        outfile <- sprintf(".significant_qtl.bonferroni_%s.%s.tsv.gz", 
                         method, filtered)
      }
      
      qtl_path <- create_output_path(output_dir, pair_base, outfile)
      fwrite(data$significant_qtls[[type]], qtl_path, sep = "\t")
    }
  }
  
  # Write n_variants stats if available
  if (!is.null(data$n_variants_stats)) {
    nv_path <- create_output_path(output_dir, regional_base, ".n_variants_stats.txt.gz")
    fwrite(data$n_variants_stats, nv_path, sep = "\t")
  }
}

#===================================
# Main Application Controller
#===================================

# A strategy interface that defines the pipeline execution strategy
create_testing_strategy <- function(local_method, global_method, global_informed_local_method, params) {
  strategy <- list(
    local_method = local_method,
    global_method = global_method,
    global_informed_local_method = global_informed_local_method,
    params = params
  )
  
  strategy$execute <- function(data) {
    result <- perform_local_adjustment(data, params, local_method)
    result <- perform_global_adjustment(result, params, global_method)
    result <- identify_significant_snps(result, params, global_informed_local_method)
    return(result)
  }
  
  return(strategy)
}

eqtl_hierarchical_testing <- function() {
  params <- parse_args()
  message(sprintf("Starting eQTL hierarchical testing in %s (QTL type: %s)", 
                  params$workdir, params$qtl_type))
  
  setwd(params$workdir)
  
  data <- list()
  data$regional_data <- load_regional_data(params)
  data$pair_data <- load_pair_data(params)
  data$gene_coords <- load_gene_coordinates(params)
  data$significant_qtls <- list()
  
  results <- data
  strategies <- list()
  
  if (!is.null(data$regional_data) && data$regional_data$has_permutation) {
    message("Setting up permutation-based testing strategies...")
    strategies$permutation_fdr_default <- create_testing_strategy("permutation", "fdr", "default", params)
    strategies$permutation_fdr_qvalue <- create_testing_strategy("permutation", "fdr", "qvalue", params)
    strategies$permutation_qvalue_default <- create_testing_strategy("permutation", "qvalue", "default", params)
    strategies$permutation_qvalue_qvalue <- create_testing_strategy("permutation", "qvalue", "qvalue", params)
  }
  
  message("Setting up Bonferroni-based testing strategies...")
  params$filtered <- FALSE
  strategies$bonferroni_fdr_default_original <- create_testing_strategy("bonferroni", "fdr", "default", params)
  strategies$bonferroni_fdr_qvalue_original <- create_testing_strategy("bonferroni", "fdr", "qvalue", params)
  strategies$bonferroni_qvalue_default_original <- create_testing_strategy("bonferroni", "qvalue", "default", params)
  strategies$bonferroni_qvalue_qvalue_original <- create_testing_strategy("bonferroni", "qvalue", "qvalue", params)
  
  if (params$qtl_type == "LR") {
    message("Setting up filtered Bonferroni-based testing strategies...")
    params$filtered <- TRUE  
    strategies$bonferroni_fdr_default_filtered <- create_testing_strategy("bonferroni", "fdr", "default", params)
    strategies$bonferroni_fdr_qvalue_filtered <- create_testing_strategy("bonferroni", "fdr", "qvalue", params)
    strategies$bonferroni_qvalue_default_filtered <- create_testing_strategy("bonferroni", "qvalue", "default", params)
    strategies$bonferroni_qvalue_qvalue_filtered <- create_testing_strategy("bonferroni", "qvalue", "qvalue", params)
  }
  
  for (name in names(strategies)) {
    message(sprintf("Executing %s strategy...", name))
    strategy_result <- strategies[[name]]$execute(data)
    
    if (!is.null(strategy_result$significant_qtls)) {
      results$significant_qtls <- c(results$significant_qtls, strategy_result$significant_qtls)
    }
    
    if (!is.null(strategy_result$n_variants_stats) && is.null(results$n_variants_stats)) {
      results$n_variants_stats <- strategy_result$n_variants_stats
    }
    
    if (strategies[[name]]$local_method == "permutation" && 
        !is.null(strategy_result$regional_data)) {
      results$regional_data <- strategy_result$regional_data
    }
    
    if (!is.null(strategy_result$global_adjustment_results)) {
      results[[paste0("global_adjustment_results_", name)]] <- 
        strategy_result$global_adjustment_results
    }
  }
  
  archive_original_files(params)
  write_results(results, params)
  
  message("eQTL hierarchical testing completed successfully")
}

# Run the application
eqtl_hierarchical_testing()